In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from opacus import PrivacyEngine
from sklearn.preprocessing import LabelEncoder


In [14]:
import pandas as pd
import numpy as np

# Load and preprocess the dataset
df = pd.read_csv("./eeg-motor-imagery-bciciv-2a/BCICIV_2a_all_patients.csv")

In [15]:
X = df.drop(columns=['label', 'patient']).values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [18]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

In [19]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [20]:
# Define the PyTorch model
class EEGNet(nn.Module):
    def __init__(self, input_dim):
        super(EEGNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(64, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        return self.fc3(x)

model = EEGNet(input_dim=X_train.shape[1])

In [21]:
# Optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()


In [22]:
# Differential Privacy using Opacus
privacy_engine = PrivacyEngine()
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=50,
    target_epsilon=5.0,
    target_delta=1e-5,
    max_grad_norm=1.0,
)


C:\Users\indir\anaconda3\lib\site-packages\opacus\privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
C:\Users\indir\anaconda3\lib\site-packages\opacus\accountants\analysis\rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


In [23]:
# Train the model
model.train()
for epoch in range(50):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

C:\Users\indir\anaconda3\lib\site-packages\torch\nn\modules\module.py:1842: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Epoch 1, Loss: 17223.3673
Epoch 2, Loss: 17212.3630
Epoch 3, Loss: 17184.0258
Epoch 4, Loss: 17212.8751
Epoch 5, Loss: 17254.8167
Epoch 6, Loss: 17301.3131
Epoch 7, Loss: 17349.5916
Epoch 8, Loss: 17411.8639
Epoch 9, Loss: 17428.1034
Epoch 10, Loss: 17470.7603
Epoch 11, Loss: 17496.1095
Epoch 12, Loss: 17525.6380
Epoch 13, Loss: 17552.6349
Epoch 14, Loss: 17601.1049
Epoch 15, Loss: 17657.5166
Epoch 16, Loss: 17686.7560
Epoch 17, Loss: 17700.8866
Epoch 18, Loss: 17712.8735
Epoch 19, Loss: 17701.8069
Epoch 20, Loss: 17742.5327
Epoch 21, Loss: 17776.3819
Epoch 22, Loss: 17830.4139
Epoch 23, Loss: 17889.6230
Epoch 24, Loss: 17944.3503
Epoch 25, Loss: 17979.6292
Epoch 26, Loss: 18043.9986
Epoch 27, Loss: 18072.7634
Epoch 28, Loss: 18141.6366
Epoch 29, Loss: 18164.6077
Epoch 30, Loss: 18205.3014
Epoch 31, Loss: 18285.8989
Epoch 32, Loss: 18330.5547
Epoch 33, Loss: 18380.2579
Epoch 34, Loss: 18403.0388
Epoch 35, Loss: 18508.6077
Epoch 36, Loss: 18506.8337
Epoch 37, Loss: 18607.0222
Epoch 38, 

In [24]:
# Save model
torch.save(model.state_dict(), "./models/hand_model_opacus.pth")